In [7]:
import prior

from integrated_agent import Agent
from leolani_client import LeolaniChatClient

ImportError: cannot import name 'Agent' from 'integrated_agent' (/Users/rvandelaar/Desktop/School/Master AI/Communicative Robots/Group project/ma-communicative-robots-group2/ai2thor/integrated_agent.py)

In [6]:
# If you need keys for openai, do so here:
with open('openaikey.txt') as f:
    api_key = f.read().strip()
openai.api_key = api_key

In [8]:
dataset = prior.load_dataset("procthor-10k")
house = dataset["train"][11]
controller = Controller(scene=house, visibilityDistance=3, width=750, height=750)

[AI2-THOR WARNING] There has been an update to ProcTHOR-10K that must be used with AI2-THOR version 5.0+. To use the new version of ProcTHOR-10K, please update AI2-THOR to version 5.0+ by running:
    pip install --upgrade ai2thor
Alternatively, to downgrade to the old version of ProcTHOR-10K, run:
   prior.load_dataset("procthor-10k", revision="ab3cacd0fc17754d4c080a3fd50b18395fae8647")


Loading test: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 2090.25it/s]


In [ ]:
# Setup names, directories
emissor_path = "./emissor"
HUMAN = "Piek"
AGENT = "Ai2Thor"

# Create the Leolani client
leolaniClient = LeolaniChatClient(emissor_path=emissor_path, agent=AGENT, human=HUMAN)

# Create your integrated Agent
agent = Agent(controller=None)  # If controller is not needed as input

# Start by asking for a human description
agent.initialize_human_description()

# Introduce the agent's capabilities
utterance = f"Hi {HUMAN}. Tell me what to do."
print(AGENT + ">" + utterance)
leolaniClient._add_utterance(AGENT, utterance)

actions_available = ", ".join(ACTIONS)
utterance = f"This is what I can do: {actions_available}"
print(AGENT + ">" + utterance)
leolaniClient._add_utterance(AGENT, utterance)

# Start conversation loop
utterance = input(HUMAN + "> ")
leolaniClient._add_utterance(HUMAN, utterance)

while not (utterance.lower() in ["stop", "bye", "exit"]):
    # Process the user instruction with the agent
    agent.process_instruction(utterance)

    # Print out and store the answers generated by the agent
    for ans in agent._answers:
        print(AGENT + ">" + ans)
        leolaniClient._add_utterance(AGENT, ans)

    # If agent perceives objects, store them with Leolani
    # agent._perceptions is a list of tuples: (obj, objectType, coord, image)
    # obj is the metadata, objectType is a string, coord a dict, image a PIL image.
    for obj, objectType, coord, image in agent._perceptions:
        # Add the perception to Leolani's memory.
        # Adjust as needed for your scenario.
        leolaniClient._add_image(obj['name'], objectType, coord, image)
    
    # If agent performed actions, store them
    # In this current code, you don't explicitly append actions to _actions, 
    # but if you did, you'd do something like:
    for action in agent._actions:
        leolaniClient._add_action(action)
    
    # Ask again
    utterance = input(HUMAN + "> ")
    leolaniClient._add_utterance(HUMAN, utterance)

# When done, stop the controller and save the scenario
agent.controller.stop()
leolaniClient._save_scenario()
print("Scenario saved and interaction ended.")